In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from simplification.cutil import (simplify_coords, simplify_coords_idx)
from timeit import default_timer as timer

In [ ]:
def local_maxima(xval, yval):
    xval = np.asarray(xval)
    yval = np.asarray(yval)

    sort_idx = np.argsort(xval)
    yval = yval[sort_idx]
    gradient = np.diff(yval)
    maxima = np.diff((gradient > 0).view(np.int8))
    return np.concatenate((([0],) if gradient[0] < 0 else ()) +
                         (np.where(maxima == -1)[0] + 1,) +
                         (([len(yval)-1],) if gradient[-1] > 0 else ()))

In [ ]:
def find_possible_toes(xval, yval):
    dx = np.diff(xval)
    dy = np.diff(yval)
    slopes = dy/dx
    w = np.diff(slopes)
    if len(w) == 0: res = np.array([0])
    else: res = w.argsort()[-5:][::-1]+1
    return res

In [ ]:
def smooth(x,window_len,window='hanning'):
    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")
    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")
    if window_len < 3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")

    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]

    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y[(window_len//2-1):-(window_len//2 + window_len % 2)]

In [ ]:
# Input file with cross-shore profiles, represented as set of points
points_all = gpd.read_file('input.gpkg')
points_all.LINE_ID.nunique()

In [ ]:
window_size = 20 # Size of Hanning window; MUST be integer
max_dist = 5 # Define max_distance between points for clustering
min_crest_Z = 2 # Min dune crest height

DP_dist = 0.1 # Min distance criteria for Douglas-Pecker line simplification
dune_height_min = 0.5 # Min height diff btw dune crest and toe
min_toe_Z = 1 # Min dune seaward toe height
toe_landward_dist = 20 # Max distance of landward toe from crest

profile = pd.DataFrame()
crests = pd.DataFrame()
toes = pd.DataFrame()
toes_landward = pd.DataFrame()

start = timer()
for i in points_all.LINE_ID.unique():
    profile = points_all[points_all.LINE_ID == i].reset_index()
    
    # Smoothing profile line
    Z = profile['Z'].values
    Z_smooth = smooth(Z, window_size)
    Z_DIST_smooth = np.vstack((Z_smooth, profile['DIST'].values)).T
    
    data = Z_DIST_smooth[:, 1], Z_DIST_smooth[:, 0]
    df_Z_DIST_smoothed = pd.DataFrame(pd.np.column_stack([pd.DataFrame(data=data).T]), columns=['DIST', 'Z'])
    
    profile_smooth = pd.merge(profile, df_Z_DIST_smoothed, on=['DIST'], how='inner', suffixes=('', '_smooth')).drop(columns=['index'])
    flex_smooth = profile_smooth[profile_smooth.index.isin(local_maxima(profile_smooth['DIST'], profile_smooth['Z_smooth']))]
    
    # Clustering smoothed flex_points
    X = flex_smooth[['DIST', 'Z_smooth']].values
    C = linkage(X, method='single', metric='euclidean') 
    clusters = fcluster(C, max_dist, criterion='distance') # Find clusters
    
    data_c = X[:, 0], X[:, 1]
    flex_smooth_clustered = pd.DataFrame(pd.np.column_stack([pd.DataFrame(data=data_c).T, clusters.T]), columns=['DIST', 'Z_smooth', 'cluster'])
    flex_smooth_merged = pd.merge(flex_smooth, flex_smooth_clustered[['DIST', 'cluster']], on=['DIST'], how='inner')
    # Search the highest points in each cluster
    poss_crest = flex_smooth_merged.groupby(['cluster'])['Z_smooth'].max().sort_values(ascending=False).reset_index()
    poss_crest = poss_crest[poss_crest['Z_smooth'] > min_crest_Z]
    crest = pd.merge(poss_crest, flex_smooth_merged, on=['Z_smooth']).sort_values(by=['DIST'])[:1].reset_index().drop(columns=['cluster_x', 'index'])
    
    # Search dune toe
    if crest.empty == False:
        for_simplify = profile_smooth[['DIST', 'Z_smooth']].values.tolist()
        profile_simplified = pd.DataFrame(simplify_coords(for_simplify, DP_dist), columns=['DIST', 'Z_smooth']) # Profile line simplification by Douglas-Pecke
        profile_cut = profile_simplified[(profile_simplified['DIST'] <= crest['DIST'][0])].reset_index()
        res_find = find_possible_toes(profile_cut['DIST'], profile_cut['Z_smooth'])
        poss_toes = profile_cut[profile_cut.index.isin(res_find)]
        order = pd.DataFrame(data=res_find, columns=['right_order'])
        poss_toes_ordered = pd.merge(order, poss_toes, how='inner', left_on='right_order', right_on='index')

        condition = (poss_toes_ordered['Z_smooth'] >= min_toe_Z) & (poss_toes_ordered['Z_smooth'] <= (crest['Z_smooth'][0]-dune_height_min))
        toe_0 = poss_toes_ordered[condition][:1]

        # Check if all possible toes are above (crest-dune_height_min)
        if toe_0.empty: toe_1 = poss_toes_ordered[:1]
        else: toe_1 = toe_0

        toe = pd.merge(toe_1, profile_smooth, on=['DIST', 'Z_smooth']).reset_index().drop(columns=['index', 'right_order', 'level_0'])
        toes = toes.append(toe)
        
        # Search landward toe
        profile_cut_landward = profile_simplified[(profile_simplified['DIST'] >= crest['DIST'][0])].reset_index()
        profile_cut_landward = profile_cut_landward.drop(columns='index')
        profile_cut_landward['index1'] = profile_cut_landward.index
        res_find_landward = find_possible_toes(profile_cut_landward['DIST'], profile_cut_landward['Z_smooth'])
        poss_toes_landward = profile_cut_landward[profile_cut_landward.index.isin(res_find_landward)]
        order_landward = pd.DataFrame(data=res_find_landward, columns=['right_order'])
        poss_toes_ordered_landward = pd.merge(order_landward, poss_toes_landward, how='inner', left_on='right_order', right_on='index1')

        toe_0_landward = poss_toes_ordered_landward[poss_toes_ordered_landward['DIST'] < crest['DIST'][0] + toe_landward_dist][:1]
        toe_landward = pd.merge(toe_0_landward, profile_smooth, on=['DIST', 'Z_smooth']).reset_index().drop(columns=['index', 'index1', 'right_order'])
        toes_landward = toes_landward.append(toe_landward)
        
    crests = crests.append(crest)
            
end = timer()
print(end - start)

In [ ]:
crests_geo = gpd.GeoDataFrame(crests, geometry=crests.geometry, crs={'init':'epsg:32637'})
toes_geo = gpd.GeoDataFrame(toes, geometry=toes.geometry, crs={'init':'epsg:32637'})
toes_landward_geo = gpd.GeoDataFrame(toes_landward, geometry=toes_landward.geometry, crs={'init':'epsg:32637'})

In [ ]:
crests_geo

In [ ]:
toes_geo

In [ ]:
toes_landward_geo

In [ ]:
crests_geo.to_file('crests.gpkg', driver='GPKG')

In [ ]:
toes_geo.to_file('toes_seaward.gpkg', driver='GPKG')

In [ ]:
toes_landward_geo.to_file('toes_landward.gpkg', driver='GPKG')